In [1]:
import pandas as pd
import os
import numpy as np
from datetime import date
#from tqdm import tqdm
import warnings
pd.set_option('display.max_rows', 2983 )
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
warnings.filterwarnings('ignore')
%pylab inline 
import seaborn as sb

Populating the interactive namespace from numpy and matplotlib


In [2]:
def get_last(df):
    return df.iloc[-1]
def get_first(df):
    return df.iloc[:1]

**CRUCE PROMOS ACTIVAS CON ATRIBUTOS DE LOS CLIENTES**

In [112]:
clients_attributes=pd.read_csv('Input/clients_attributes.csv')
clients_attributes.head() # Table maestro de clientes

,Cliente,FechaAltaCliente,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF
0,1,2009-03-03,4,21,12,1,1,1
1,9,2009-03-03,4,21,9,2,3,1
2,15,2009-03-03,4,21,9,2,3,0
3,37,2009-03-03,4,21,16,2,3,1
4,43,2009-03-03,4,21,9,2,3,0


In [108]:
clients_attributes.shape

(15069, 8)

In [113]:
active_promos=pd.read_csv('Input/active_promos.csv')
active_promos.head()

,CodigoDC,Marca,Cupo,Fecha_Desde,Fecha_Hasta,Cliente
0,297601,29,9,2018-08-06,2018-08-30,3213
1,297601,29,9,2018-08-06,2018-08-30,3795
2,297601,29,9,2018-08-06,2018-08-30,11816
3,298856,40,16,2018-08-12,2018-08-19,8444
4,309916,40,16,2018-08-12,2018-08-19,8488


In [122]:
active_promos.Fecha_Hasta.max()

'2019-10-13'

In [114]:
active_promos.shape

(422437, 6)

In [159]:
promos_activas=pd.merge(active_promos, clients_attributes, how='left', on=['Cliente'])
promos_activas['Fecha_Desde']=pd.to_datetime(promos_activas['Fecha_Desde'])
promos_activas['Fecha_Hasta']=pd.to_datetime(promos_activas['Fecha_Hasta'])
promos_activas['FechaAltaCliente']=pd.to_datetime(promos_activas['FechaAltaCliente'])
promos_activas['key']=promos_activas['CodigoDC'].astype('str')+promos_activas['Cliente'].astype('str')+promos_activas['Marca'].astype('str')+promos_activas['Cupo'].astype('str')
promos_activas.head() # Base de todas las promociones dispuestas para los clientes

,CodigoDC,Marca,Cupo,Fecha_Desde,Fecha_Hasta,Cliente,FechaAltaCliente,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,key
0,297601,29,9,2018-08-06,2018-08-30,3213,2010-03-29,3,8,15,2,3,1,2976013213299
1,297601,29,9,2018-08-06,2018-08-30,3795,2010-04-26,3,10,6,2,3,1,2976013795299
2,297601,29,9,2018-08-06,2018-08-30,11816,2017-10-01,3,7,12,2,3,1,29760111816299
3,298856,40,16,2018-08-12,2018-08-19,8444,2015-09-02,5,27,18,2,2,1,29885684444016
4,309916,40,16,2018-08-12,2018-08-19,8488,2015-09-16,3,10,16,2,2,0,30991684884016


In [195]:
promos_activas.dtypes

CodigoDC                     int64
Marca                        int64
Cupo                         int64
Fecha_Desde         datetime64[ns]
Fecha_Hasta         datetime64[ns]
Cliente                      int64
FechaAltaCliente    datetime64[ns]
Region                       int64
Gerencia                     int64
SubCanal                     int64
TipoPoblacion                int64
Estrato                      int64
EF                           int64
key                         object
dtype: object

**CRUCE DE TODAS LAS PROMOCIONES DADAS CON LAS PROMOCIONES EJECUTADAS**

In [160]:
promos_ejecutadas=pd.read_csv('Input/executed_promos.csv')
promos_ejecutadas['key']=promos_ejecutadas['CodigoDC'].astype('str')+promos_ejecutadas['Cliente'].astype('str')+promos_ejecutadas['Marca'].astype('str')+promos_ejecutadas['Cupo'].astype('str')
promos_ejecutadas['Target']=1
promos_ejecutadas

,CodigoDC,Cliente,Marca,Cupo,key,Target
0,297601,8410,29,9,2976018410299,1
1,297601,9337,29,9,2976019337299,1
2,309843,7404,40,16,30984374044016,1
3,309874,4017,40,16,30987440174016,1
4,309910,6316,40,16,30991063164016,1
...,...,...,...,...,...,...
27789,651436,9798,39,20,65143697983920,1
27790,651588,2397,39,20,65158823973920,1
27791,651597,4430,40,16,65159744304016,1
27792,651617,7347,39,20,65161773473920,1


In [276]:
df1=pd.merge(promos_activas, promos_ejecutadas[['key', 'Target']], how='left', on=['key'])
df1['Target']=df1['Target'].fillna(0).astype('int')
df1['month_desde']=pd.DatetimeIndex(df1['Fecha_Desde']).month
df1['month_hasta']=pd.DatetimeIndex(df1['Fecha_Hasta']).month
df1['year_desde']=pd.DatetimeIndex(df1['Fecha_Desde']).year
df1['year_hasta']=pd.DatetimeIndex(df1['Fecha_Hasta']).year
df1['resta_month']=df1['month_desde']-df1['month_hasta']
df1['resta_year']=df1['year_desde']-df1['year_hasta']
df1['key2']=df1['Cliente'].astype('str')+df1['Marca'].astype('str')+df1['Cupo'].astype('str')+df1['year_desde'].astype('str')+df1['month_desde'].astype('str')
df1.head(1500)

,CodigoDC,Marca,Cupo,Fecha_Desde,Fecha_Hasta,Cliente,FechaAltaCliente,Region,Gerencia,SubCanal,TipoPoblacion,Estrato,EF,key,Target,month_desde,month_hasta,year_desde,year_hasta,resta_month,resta_year,key2
0,297601,29,9,2018-08-06,2018-08-30,3213,2010-03-29,3,8,15,2,3,1,2976013213299,0,8,8,2018,2018,0,0,321329920188
1,297601,29,9,2018-08-06,2018-08-30,3795,2010-04-26,3,10,6,2,3,1,2976013795299,0,8,8,2018,2018,0,0,379529920188
2,297601,29,9,2018-08-06,2018-08-30,11816,2017-10-01,3,7,12,2,3,1,29760111816299,1,8,8,2018,2018,0,0,1181629920188
3,298856,40,16,2018-08-12,2018-08-19,8444,2015-09-02,5,27,18,2,2,1,29885684444016,1,8,8,2018,2018,0,0,8444401620188
4,309916,40,16,2018-08-12,2018-08-19,8488,2015-09-16,3,10,16,2,2,0,30991684884016,1,8,8,2018,2018,0,0,8488401620188
5,309962,40,16,2018-08-12,2018-08-24,7159,2014-07-09,3,10,12,2,1,1,30996271594016,1,8,8,2018,2018,0,0,7159401620188
6,310695,40,16,2018-08-12,2018-08-19,2218,2010-01-31,3,25,16,2,2,1,31069522184016,1,8,8,2018,2018,0,0,2218401620188
7,311170,40,16,2018-08-12,2018-08-22,7189,2014-07-21,3,28,16,1,2,1,31117071894016,1,8,8,2018,2018,0,0,7189401620188
8,311276,40,16,2018-08-12,2018-08-19,1644,2009-11-09,5,27,16,2,1,1,31127616444016,1,8,8,2018,2018,0,0,1644401620188
9,311283,40,16,2018-08-12,2018-08-21,7114,2014-06-25,5,27,16,2,2,0,31128371144016,1,8,8,2018,2018,0,0,7114401620188


In [291]:
df1.to_csv('Output/df1.csv')

In [277]:
sales=pd.read_csv('Input/sales.csv')
sales.loc[(sales.Anio==2018) & (sales.Mes==1),'Flag']=0
sales.loc[(sales.Anio==2018) & (sales.Mes==2),'Flag']=0
sales.loc[(sales.Anio==2018) & (sales.Mes==3),'Flag']=0
sales.loc[(sales.Anio==2018) & (sales.Mes==4),'Flag']=0
sales.loc[(sales.Anio==2018) & (sales.Mes==5),'Flag']=0
sales.loc[(sales.Anio==2018) & (sales.Mes==6),'Flag']=0
sales.loc[(sales.Anio==2018) & (sales.Mes==7),'Flag']=0
sales['Flag']=sales['Flag'].fillna(1)
sales=sales[sales.Flag==1]
sales['key2']=sales['Cliente'].astype('str')+sales['Marca'].astype('str')+sales['Cupo'].astype('str')+sales['Anio'].astype('str')+sales['Mes'].astype('str')
sales

,Cliente,Anio,Mes,ClaseEnvase,SegmentoPrecio,Marca,Cupo,Nr,Hl,Dcto,Flag,key2
4,1,2018,8,2,1,2,16,16250.18,0.06,0.00,1.0,121620188
5,1,2018,9,1,2,44,30,8901.19,0.05,0.00,1.0,1443020189
6,1,2018,9,1,2,44,32,16944.77,0.05,-1898.32,1.0,1443220189
7,1,2018,10,1,2,44,32,12551.68,0.04,0.00,1.0,14432201810
8,1,2018,10,2,1,6,12,93730.38,0.31,0.00,1.0,1612201810
...,...,...,...,...,...,...,...,...,...,...,...,...
1048565,5036,2019,5,1,2,37,30,2678.71,0.03,-1794.96,1.0,5036373020195
1048566,5036,2019,5,1,2,44,29,7729.12,0.04,-323.70,1.0,5036442920195
1048567,5036,2019,6,1,2,44,31,5116.63,0.04,0.00,1.0,5036443120196
1048568,5036,2019,6,1,3,24,25,2535.52,0.01,0.00,1.0,5036242520196


In [ ]:
sales.to_csv('Output/sales2.csv')

# INPUT:

In [220]:
test=pd.read_csv('Input/test.csv')
test['key']=test['Cliente'].astype('str')+test['Marca'].astype('str')+test['Cupo'].astype('str')
test

,Cliente,Marca,Cupo,Ejecuto_Promo,key
0,10,40,16,NaN,104016
1,17,29,9,NaN,17299
2,37,29,9,NaN,37299
3,46,29,9,NaN,46299
4,48,29,9,NaN,48299
...,...,...,...,...,...
16865,15053,40,16,NaN,150534016
16866,15054,40,16,NaN,150544016
16867,15056,40,16,NaN,150564016
16868,15057,29,9,NaN,15057299


In [219]:
test.shape

(16870, 4)

In [221]:
test.key.unique().shape

(16870,)